# FAISS VectorStore
1. CSV에서 한 행씩 chunking

1. CSVLoader() 로 csv 파일 로드
2. xml document 생성
3. TextSplitter로 분할 : chunk_size=600, 
4. chroma db 생성(우선 10개만)
   - splited text로 db 생성
   - embedding : OpenAIEmbedding()
5. query 테스트

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH23-MyProject")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH23-MyProject


In [3]:
import os
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

In [4]:
# CSV(공고데이터) 로드
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 경로
loader = CSVLoader(
    file_path="./data/announcement.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        # "quotechar": '"',  # 인용 부호 문자
        # "fieldnames": [
        #     "Sequence",  # 순번
        #     "Registration number",  # 공고번호
        #     "Announcement name",  # 공고명
        #     "Support areas",  # 지원분야
        #     "Region",  # 지역
        #     "Target",  # 지원대상
        #     "Target age",  # 대상연령
        #     "Application period",  # 접수기간
        #     "Entrepreneurial history",  # 업력
        #     "Institution name",  # 기관명
        #     "Organization classification",  # 기관구분 : 공공, 민간, 교육
        #     "Department in charge",  # 담당부서
        #     "Announcement number",  # 공고 제 호
        #     "Announcement contents",  # 공고내용
        #     "Registration date",  # 공고등록일
        #     "Announcement registrar name",  # 공고 기업명
        #     "How to apply work-in",  # 신청방법 : 방문
        #     "How to apply By mail",  # 신청방법 : 우편
        #     "How to apply By Fax",  # 신청방법 : FAX
        #     "How to apply By email",  # 신청방법 : email
        #     "How to apply online",  # 신청방법 : 온라인
        #     "How to apply other",  # 신청방법 : 기타
        #     "Who to apply for",  # 신청대상
        #     "Excluded from application",  # 제외대상
        #     "Summary",  # 공고명 + 공고내용
        # ],  # 필드 이름
    },
)

# 데이터 로드
docs = loader.load()

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "mps"},  # cuda, cpu, mps
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
from langchain_community.vectorstores import FAISS

# 첫 번째 행으로 FAISS DB 초기화
first_doc = next(loader.lazy_load())
db = FAISS.from_documents(documents=[first_doc], embedding=hf_embeddings)

# 나머지 행들을 한 줄씩 처리하여 추가
for doc in loader.lazy_load():
    db.add_documents([doc])

# FAISS 인덱스 저장 (선택사항)
db.save_local("faiss_hfe_index")

In [12]:
def search_top_similar(db, query, top_k=3):
    # 유사도 검색 실행
    results = db.similarity_search_with_score(query=query, k=top_k)

    # score 기준으로 정렬 (낮은 점수가 더 유사함)
    sorted_results = sorted(results, key=lambda x: x[1], reverse=False)

    # 상위 3개 결과 출력
    print(f"\n검색어: {query}")
    print("\n=== 가장 유사한 상위 3개 결과 ===")

    for idx, (doc, score) in enumerate(sorted_results[:top_k], 1):
        print(f"\n{idx}위 (유사도 점수: {score:.4f})")
        print(f"내용: {doc.page_content}")
        if hasattr(doc, "metadata") and doc.metadata:
            print(f"메타데이터: {doc.metadata}")
        print("-" * 50)

    return sorted_results[:top_k]


# 사용 예시
db = FAISS.load_local(
    "faiss_hfe_index", hf_embeddings, allow_dangerous_deserialization=True
)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings


def search_top_similar2(db, query, top_k=3):
    results = db.similarity_search_with_score(query=query, k=top_k)

    sorted_results = sorted(results, key=lambda x: x[1])

    print(f"\n검색어: {query}")
    print("\n=== 가장 유사한 상위 3개 결과 ===")

    for idx, (doc, score) in enumerate(sorted_results[:top_k], 1):
        # page_content에서 공고명 추출 (형식에 따라 수정 필요)
        content = doc.page_content
        if isinstance(content, str):
            # 문자열에서 공고명 필드 찾기
            if "공고명:" in content:
                notice_name = content.split("공고명:")[1].split("\n")[0].strip()
            else:
                notice_name = "공고명 찾을 수 없음"
        elif isinstance(content, dict):
            # 딕셔너리에서 공고명 필드 찾기
            notice_name = content.get("공고명", "공고명 없음")
        else:
            notice_name = "지원되지 않는 형식"

        print(f"\n{idx}위 (유사도 점수: {score:.4f})")
        print(f"공고명: {notice_name}")
        print("-" * 50)

    return sorted_results[:top_k]

In [14]:
# 검색 실행
#query = "군인을 위한 창업 프로그램은"
query = "예비창업패키지가 뭔가요"
top_results = search_top_similar(db, query, top_k=3)


검색어: 예비창업패키지가 뭔가요

=== 가장 유사한 상위 3개 결과 ===

1위 (유사도 점수: 0.2110)
내용: ﻿순번: 223
공고번호: 167495
공고명: 2024년도 예비창업패키지 예비창업자 모집 공고(수정)
지원분야: 사업화
지역: 전국
대상: 일반인
대상연령: 전체
접수기간: 2024-01-30 ~ 2024-02-21 16:00
창업업력: 예비창업자
기관명: 창업진흥원
기관구분: 공공기관
담당부서: 예비초기창업실
공고제호: 
공고내용: 혁신적인 기술과 사업모델(BM)을 보유한 예비창업자를 발굴ㆍ지원하기 위한
『2024년 예비창업패키지』 참여자를 다음과 같이 모집합니다.

* 수정사항 : 공고문 "[붙임3] 중소벤처기업부 창업사업화 지원 제외사업 목록" 중 지원 제외사업(1) 추가(2023년 사내벤처 육성 프로그램)
공고등록일: 2024년 1월 30일
공고기업명: 중소벤처기업부
신청방법-방문접수: 
신청방법-우편접수: 
신청방법-팩스접수: 
신청방법-이메일접수: 
신청방법-온라인접수: https://www.k-startup.go.kr/web/contents/webPMSBizUnvs.do?returnUrl=/screen/BMO0101M01?pbancId=0566801
신청방법-기타: 
신청대상: 아래 신청자격을 모두 충족한 자
- 사업공고일(‘24.1.30.) 기준 신청자 명의의 사업자 등록 및 법인설립 등기를 하지  없는 자
- 사업공고일(‘24.1.30.) 기준 최근 1개월 이내('24.1.1.~1.30.)에 개인사업자 폐업 또는 법인사업자 폐업(해산 및 청산)을 하지 않은 자
※ 상세 자격요건은 공고문 참조
제외대상: 공고문 신청 제외 대상 참조
요약: 2024년도 예비창업패키지 예비창업자 모집 공고(수정)  혁신적인 기술과 사업모델(BM)을 보유한 예비창업자를 발굴ㆍ지원하기 위한
『2024년 예비창업패키지』 참여자를 다음과 같이 모집합니다.

* 수정사항 : 공고문 "[붙임3] 중소벤처기업부 창업사업화 지원 제외사업 목록" 중 지원 제

In [20]:
# FAISS index 불러오기
faiss_index_path = "./faiss_1_row_index"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

faiss_db = FAISS.load_local(
    faiss_index_path, embeddings, allow_dangerous_deserialization=True
)

In [ ]:
query1 = "서울지역에서 하는 창업관련 교육 프로그램 알려줘"
query2 = "창업진흥원에서 하는 초기창업패키지 모집기간이 언제야"
result = faiss_db.similarity_search_with_score(query1, k=3)
# result = faiss_db.similarity_search_with_score(query2, k=3)

for doc, score in result:
    print(f"문서내용: {doc.page_content}")
    print(f"유사도점수: {score}")
    print("---------------------------------------------------")

![](./images/chroma-06.png)